**NARGEOT** Melvine
**PATTE** Mathys

# Projet Outil Informatique

https://www.sciencedirect.com/topics/earth-and-planetary-sciences/volcanic-aerosol

The Greenhouse Effect, Aerosols, and Climate Change

Daniel Kirk-Davidoff, in Green Chemistry, 2018

"Volcanoes emit sulfur dioxide gas (SO2), which reacts with water in the atmosphere to form sulfuric acid (H2SO4). When volcanic plumes are emitted powerfully enough to reach the stratosphere,a the H2SO4 can form a persistent haze of liquid droplets, reflecting away sunlight and cooling the earth for a year or two."

In [ ]:
%pip install numpy matplotlib pandas # Handle data
%pip install xarray netcdf4 h5netcdf # Handle NetCFD file
%pip install folium # World map
%pip install beautifulsoup4 # Web scrapping
%pip install geopandas
%pip install slugify

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import folium as fl
from bs4 import BeautifulSoup
from urllib.request import urlopen
import geopandas as gpd
import slugify

## Données

### Sources

https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/volcanoes?page=1

https://volcano.si.edu/search_eruption.cfm

https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions/

https://data.cerdi.uca.fr/erup-vol/

https://public.opendatasoft.com/explore/dataset/significant-volcanic-eruption-database/export/

https://www.ngdc.noaa.gov/hazel/view/hazards/volcano/event-search/

https://berkeleyearth.org/data/

https://disc.gsfc.nasa.gov/datasets?keywords=%22msvolso2l4%22%7C%22tomsn7so2%22%7C%22OMPS_NPP_NMSO2_PCA_L2%22%7C%22MSAQSO2L4%22&page=1&subject=Aerosols

https://measures.gesdisc.eosdis.nasa.gov/data/SO2/MSVOLSO2L4.4/MSVOLSO2L4_v04-00-2022m0505.txt

### Import

In [64]:
from urllib.request import urlopen
from io import StringIO

# https://measures.gesdisc.eosdis.nasa.gov/data/SO2/MSVOLSO2L4.4/MSVOLSO2L4_v04-00-2022m0505.txt
data = urlopen("https://melvin-klein.github.io/outinfo/so2.txt").readlines()[47:]

# Some lines have too many cells, we delete them.
data[1172] = data[1172][:-4] + b'\n'
data[2033] = data[2033][:-4] + b'\n'

data = str(b''.join(data), 'utf-8')

data = StringIO(data) 

so2 = pd.read_csv(data, sep="\t")

so2

,volcano,lat,lon,v_alt,yyyy,mm,dd,type,vei,p_alt_obs,p_alt_est,so2(kt),Unnamed: 12
0,Ambrym,-16.25,168.12,1.334,1978.0,12.0,5.0,exp,2.0,-999.0,11.334,87.0,NaN
1,Cerro_Azul,-0.92,-91.408,1.690,1979.0,2.0,1.0,eff,2.0,2.0,-999.000,10.0,NaN
2,Cerro_Azul,-0.92,-91.408,1.690,1979.0,2.0,2.0,eff,2.0,7.0,-999.000,60.0,NaN
3,Cerro_Azul,-0.92,-91.408,1.690,1979.0,2.0,3.0,eff,2.0,2.0,-999.000,40.0,NaN
4,Cerro_Azul,-0.92,-91.408,1.690,1979.0,2.0,4.0,eff,2.0,3.0,-999.000,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,Manam -4.1 145.061 1.807 2022 4 ...,-999 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2199,Karymsky,54.05,159.43,1.486,2022.0,4.0,20.0,exp,3.0,10.0,-999.000,8.0,NaN
2200,Krakatau -6.102 105.423 0.813 2022 4 ...,-999 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2201,Krakatau -6.102 105.423 0.813 2022 4 ...,-999 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
columns = ['volcano', 'lat', 'lon', 'yyyy', 'mm', 'so2(kt)']

so2_cleared = so2.dropna(subset=columns)

so2_cleared = so2_cleared[columns]

so2_cleared

,volcano,lat,lon,yyyy,mm,so2(kt)
0,Ambrym,-16.25,168.12,1978.0,12.0,87.0
1,Cerro_Azul,-0.92,-91.408,1979.0,2.0,10.0
2,Cerro_Azul,-0.92,-91.408,1979.0,2.0,60.0
3,Cerro_Azul,-0.92,-91.408,1979.0,2.0,40.0
4,Cerro_Azul,-0.92,-91.408,1979.0,2.0,10.0
...,...,...,...,...,...,...
2192,Chikurachki,50.325,155.458,2022.0,1.0,7.0
2193,Etna,37.734,15.004,2022.0,2.0,15.0
2194,Etna,37.734,15.004,2022.0,2.0,17.0
2197,Bezymianny,55.978,160.587,2022.0,3.0,5.0


In [66]:
# Add countries to volcano

gdf_eruptions = gpd.read_file("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/significant-volcanic-eruption-database/exports/geojson?lang=fr&timezone=Europe%2FBerlin")

gdf_eruptions_cleared = gdf_eruptions[columns]

gdf_eruptions_cleared = gdf_eruptions_cleared.dropna(subset=columns)

gdf_eruptions_cleared

,name,country,elevation,type,year,month,geometry
2,Hekla,Iceland,1491,Stratovolcano,1300,07,POINT (-19.70000 63.98000)
3,Aso,Japan,1592,Caldera,1331,12,POINT (131.10000 32.88000)
4,Vesuvius,Italy,1281,Complex volcano,1714,06,POINT (14.42600 40.82100)
5,Savai'i,Samoa,1858,Shield volcano,1907,10,POINT (-172.52500 -13.61200)
6,Asama,Japan,2560,Complex volcano,1911,08,POINT (138.53000 36.40000)
...,...,...,...,...,...,...,...
830,Etna,Italy,3350,Stratovolcano,1979,07,POINT (15.00400 37.73400)
831,Galunggung,Indonesia,2168,Stratovolcano,1982,05,POINT (108.05000 -7.25000)
832,Arenal,Costa Rica,1657,Stratovolcano,1988,07,POINT (-84.70300 10.46300)
833,Karthala,Comoros,2361,Shield volcano,2005,11,POINT (43.38000 -11.75000)


## Try to merge the two Geopandas dataframe

In [67]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(so2_cleared['lon'], so2_cleared['lat'])]
gdf_so2_cleared = gpd.GeoDataFrame(so2_cleared, geometry=geometry, crs='EPSG:4326')
gdf_so2_cleared

,volcano,lat,lon,yyyy,mm,so2(kt),geometry
0,Ambrym,-16.25,168.12,1978.0,12.0,87.0,POINT (168.12000 -16.25000)
1,Cerro_Azul,-0.92,-91.408,1979.0,2.0,10.0,POINT (-91.40800 -0.92000)
2,Cerro_Azul,-0.92,-91.408,1979.0,2.0,60.0,POINT (-91.40800 -0.92000)
3,Cerro_Azul,-0.92,-91.408,1979.0,2.0,40.0,POINT (-91.40800 -0.92000)
4,Cerro_Azul,-0.92,-91.408,1979.0,2.0,10.0,POINT (-91.40800 -0.92000)
...,...,...,...,...,...,...,...
2192,Chikurachki,50.325,155.458,2022.0,1.0,7.0,POINT (155.45800 50.32500)
2193,Etna,37.734,15.004,2022.0,2.0,15.0,POINT (15.00400 37.73400)
2194,Etna,37.734,15.004,2022.0,2.0,17.0,POINT (15.00400 37.73400)
2197,Bezymianny,55.978,160.587,2022.0,3.0,5.0,POINT (160.58700 55.97800)


In [73]:
joined_gdf = gpd.sjoin(gdf_so2_cleared, gdf_eruptions_cleared, how='left', op='intersects')
filtered_gdf = joined_gdf[joined_gdf['yyyy'] == joined_gdf['year']]
filtered_gdf

/home/melvine/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,volcano,lat,lon,yyyy,mm,so2(kt),geometry,index_right,name,country,elevation,type,year,month
13,Karkar,-4.649,145.964,1979.0,3.0,50.0,POINT (145.96400 -4.64900),81.0,Karkar,Papua New Guinea,1839.0,Stratovolcano,1979.0,03
14,Soufriere_St_Vincent,13.33,-61.18,1979.0,4.0,3.0,POINT (-61.18000 13.33000),13.0,Soufriere St. Vincent,St. Vincent & the Grenadines,1220.0,Stratovolcano,1979.0,04
15,Etna,37.734,15.004,1979.0,8.0,10.0,POINT (15.00400 37.73400),830.0,Etna,Italy,3350.0,Stratovolcano,1979.0,07
63,Mt_St_Helens,46.2,-122.18,1980.0,5.0,775.0,POINT (-122.18000 46.20000),709.0,St. Helens,United States,2549.0,Stratovolcano,1980.0,05
64,Mt_St_Helens,46.2,-122.18,1980.0,5.0,100.0,POINT (-122.18000 46.20000),709.0,St. Helens,United States,2549.0,Stratovolcano,1980.0,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,Ulawun,-5.05,151.33,2019.0,6.0,150.0,POINT (151.33000 -5.05000),194.0,Ulawun,Papua New Guinea,2334.0,Stratovolcano,2019.0,06
1941,Manam,-4.1,145.061,2019.0,6.0,10.0,POINT (145.06100 -4.10000),807.0,Manam,Papua New Guinea,1807.0,Stratovolcano,2019.0,01
1941,Manam,-4.1,145.061,2019.0,6.0,10.0,POINT (145.06100 -4.10000),721.0,Manam,Papua New Guinea,1807.0,Stratovolcano,2019.0,06
1945,Ulawun,-5.05,151.33,2019.0,8.0,190.0,POINT (151.33000 -5.05000),194.0,Ulawun,Papua New Guinea,2334.0,Stratovolcano,2019.0,06


## End of try

In [ ]:
volcano_weather = pd.DataFrame(columns=['year', 'month', 'volcano_name', 'country', 'eruption_so2', 'lat', 'lng'])

In [ ]:
volcano_weather['year'] = pd.to_datetime(so2_cleared['yyyy'], format="%Y").dt.year
volcano_weather['month'] = pd.to_datetime(so2_cleared['mm'], format="%m").dt.month
volcano_weather[['volcano_name', 'eruption_so2', 'lat', 'lng']] = so2_cleared[['volcano', 'so2(kt)', 'lat', 'lon']]

volcano_weather['id'] = volcano_weather['volcano_name'].apply(lambda x: slugify.slugify(x))

volcano_id_customdf = sorted(volcano_weather[volcano_weather['id'].isin(volcano_id_geojson)]['id'].unique().tolist())

len(volcano_weather['volcano_name'].unique().tolist())

volcano_weather

In [ ]:
volcano_id_geojson

In [ ]:
set(volcano_weather['id'].unique().tolist()) - set(volcano_id_geojson)

In [ ]:
volcano_map = fl.Map(zoom_start=10)

for index, row in volcano_weather.drop_duplicates(subset='volcano_name', keep='first').iterrows():
    fl.Marker(
        location=[row['lat'], row['lng']],
        popup=row['volcano_name']
    ).add_to(volcano_map)

volcano_map

In [51]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s3_en.php?block_no=47401"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [52]:
rows = soup.find_all("tr", {"class": "mtx"})

for row in rows:
    data = row.get_text().split(' ')

rows[1].get_text(" ").split(' ')

['1939',
 '-7.5',
 '-6.6',
 '-1.4',
 '4.0',
 '7.5',
 '13.0',
 '17.4',
 '20.0',
 '17.4',
 '9.7',
 '3.0',
 '-2.5',
 '6.2']

## Analyse

In [ ]:
monthly_t = pd.read_json("https://pkgstore.datahub.io/core/global-temp/monthly_json/data/4c7af7363a20648a68b8f2038a6765d6/monthly_json.json") # Reading the json file of global monthly temperature anomaly

monthly_t = monthly_t.loc[monthly_t["Source"] == "GISTEMP"] # Only keep values from GISTEMP (nasa)
monthly_t.reset_index(inplace=True) # Reset row index to manipulate rows easily

# Splitting the date in format yyyy-mm-dd into Year and Month to use it with other data later
year = [monthly_t["Date"][date].year for date in range(len(monthly_t["Date"]))] # Create a 'Year' column from the timestamp given in 'Date'
monthly_t["year"] = year

month = [monthly_t["Date"][date].month for date in range(len(monthly_t["Date"]))] # Create a 'Month' column from the timestamp given in 'Date'
monthly_t["month"] = month

monthly_t = monthly_t.drop(['Date', 'Source'], axis=1) # Delete useless rows for later use
monthly_t = monthly_t.reindex(columns=['year', 'month', 'Mean']) # Put date first then temperature anomaly
monthly_t.sort_values(by=['year','month'], inplace=True) # Sort by ascending date
monthly_t.reset_index(inplace=True, drop=True) # Reset index for readability 
monthly_t = monthly_t.rename(columns={"Mean" : "t_anomaly"}) # rename Mean for readability
monthly_t


In [ ]:
monthly_t.reset_index(inplace=True, drop=True)
volcano_weather.reset_index(inplace=True, drop=True)

date = [str(volcano_weather["year"][date]) + str(volcano_weather["month"][date]) for date in range(len(volcano_weather["year"]))]
volcano_weather["date"] = date
date = [str(monthly_t["year"][date]) + str(monthly_t["month"][date]) for date in range(len(monthly_t["year"]))]
monthly_t["Date"] = date

monthly_t = monthly_t[monthly_t['Date'].isin(volcano_weather["date"])]


volcano_temperature = pd.merge(monthly_t, volcano_weather, on=["year", "month"], how="right")
print(volcano_temperature)


In [ ]:
plt.figure(figsize=(9,4), dpi=100)
plt.plot(volcano_temperature["year"], volcano_temperature["t_anomaly"])
plt.xlabel('Year')
plt.ylabel('Temperature anomaly')
plt.legend(['Anomaly of temperature for each year'], loc=(0.02, 0.88))
plt.title('Comparison of temperature and sulfure emission', pad=10, fontsize=14, style='italic')
plt.subplot()
plt.plot(volcano_temperature["year"], volcano_temperature["eruption_so2"]/10000); plt.legend("Sulfure emission for each year")

In [ ]:
plt.figure(figsize=(8,4), dpi=100)
plt.plot(volcano_temperature["t_anomaly"], np.log(volcano_temperature["eruption_so2"]))
plt.xlabel('temperature anomaly'); plt.ylabel('sulfure emission'); # labels
plt.title('Visualisation de données éparses', pad=10, fontsize=14, style='italic'); # titre